# Example of PCA using the correlation matrix

Author: Edrei Santos

In [3]:
# Importing data and libraries
import numpy as np
import polars as pl
import scipy as sp

data = pl.read_excel("dados_brutos_cap3_mingoti.xlsx")
data = data.drop(data.columns[0])
data = data.to_numpy()

print(data)

[[ 9893   564 17689]
 [ 8776   389 17359]
 [13572  1103 18597]
 [ 6455   743  8745]
 [ 5129   203 14397]
 [ 5432   215  3467]
 [ 3807   385  4679]
 [ 3423   187  6754]
 [ 3708   127  2275]
 [ 3294   297  6754]
 [ 5433   432  5589]
 [ 6287   451  8972]]


I will now check if the correlation matrix of the data pass a T-test at 5% of significance, or α = 0.05.

In [4]:
corr_matrix = np.corrcoef(data, rowvar= False)
print("Correlation Matrix \n\n",corr_matrix)
p_values = np.empty((3,3))
for j in range(data.shape[1]):
    for i in range(data.shape[1]):
        temp = sp.stats.pearsonr(data[:,j], data[:,i])
        p_values[j,i] = temp[1]
print("\n P Values Matrix \n\n",p_values)

Correlation Matrix 

 [[1.         0.82734786 0.82625598]
 [0.82734786 1.         0.57650285]
 [0.82625598 0.57650285 1.        ]]

 P Values Matrix 

 [[0.         0.00089716 0.00092409]
 [0.00089716 0.         0.04974535]
 [0.00092409 0.04974535 0.        ]]


As we can see, all Correlation Values passed the α = 0.05 significance test. Next step is to get the eigvectors and values of the correlation matrix.

In [5]:
eigval , eigvec = np.linalg.eig(corr_matrix)
print("Eigenvalues: ", eigval)
print("Eigenvectors: \n", eigvec)

Eigenvalues:  [2.49253211 0.08396976 0.42349813]
Eigenvectors: 
 [[ 0.61670267  0.78719515 -0.00126721]
 [ 0.55679445 -0.43733949 -0.70619694]
 [ 0.556469   -0.43480796  0.70801432]]


The selection method of getting the components is choosing the vectors that have values higher than 1. So we will only use the component of highest eigenvalue (2.5).

Before we apply the vector to the dataset, we need to standardize the numbers of the matrix

In [6]:
data_means = np.mean(data,0)
data_std = np.std(data,0, ddof= 1)
data = (data - data_means) / data_std
print(data)

[[ 1.17317383  0.50452133  1.37790655]
 [ 0.81173264 -0.12914781  1.32164865]
 [ 2.36363234  2.45622228  1.53270101]
 [ 0.06069861  1.15267434 -0.14685301]
 [-0.36837124 -0.80264757  0.81669138]
 [-0.27032587 -0.75919598 -1.04663844]
 [-0.79614677 -0.14363167 -0.84001852]
 [-0.92040229 -0.86058304 -0.48627567]
 [-0.82818139 -1.07784103 -1.24984879]
 [-0.96214438 -0.46227672 -0.48627567]
 [-0.27000229  0.02655375 -0.6848831 ]
 [ 0.00633682  0.09535212 -0.10815439]]


Now that the data is standardized, I can apply the eigenvector of the component in the data.

In [7]:
data = data @ eigvec[:,0]
print(data)

[ 1.77117640e+00  1.16414541e+00  3.67816992e+00  5.97516523e-01
 -2.19621809e-01 -1.17184865e+00 -1.03840342e+00 -1.31737975e+00
 -1.80637970e+00 -1.12134746e+00 -5.32842366e-01 -3.18510461e-03]


In [8]:
df = {"Company": range(data.shape[0]), "Scores": data}
data = pl.DataFrame(df)
data = data.with_columns(pl.col("Company") + 1)
data.sort("Scores", descending= True)

Company,Scores
i64,f64
3,3.67817
1,1.771176
2,1.164145
4,0.597517
12,-0.003185
…,…
7,-1.038403
10,-1.121347
6,-1.171849


By this method, company 3 is the best, and company 9 is the worst.